# Extraire les intensités des lesions en fonction des labels


## Charger le T1 (P1_T1_T1.nii.gz) et le masque de lésion (result.nii.gz).
## Charger le masque ASEG (par exemple aseg.nii.gz) pour les zones du cerveau.
## Exclure les zones que tu ne veux pas (ventricules + LCR cortical).
## Identifier chaque lésion comme un label distinct.
## Pour chaque lésion :
## Calculer moyenne et std sur toute la lésion.
## Pour chaque zone ASEG, calculer moyenne et std dans l’intersection lésion × zone.
## Sauvegarder un CSV avec une ligne par lésion et toutes les moyennes par zone.

In [6]:
import SimpleITK as sitk
import numpy as np
import csv

# -----------------------------
# 1) Chemins des fichiers
# -----------------------------
aseg_path = "../../27919209/MSLesSegDataset/train/P1/T1/recalage.nii.gz"
mask_path = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_MASK.nii.gz"
t1_path   = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T1.nii.gz"

# -----------------------------
# 2) Lecture des images
# -----------------------------
t1_img   = sitk.Cast(sitk.ReadImage(t1_path), sitk.sitkFloat64)
mask_img = sitk.Cast(sitk.ReadImage(mask_path) > 0, sitk.sitkUInt8)
aseg_img = sitk.ReadImage(aseg_path)

# -----------------------------
# 3) Labels ASEG à exclure
# -----------------------------
exclude_labels = [4, 43, 14, 15, 24, 31, 63]

# -----------------------------
# 4) Identifier les lésions distinctes
# -----------------------------
mask_labels = sitk.ConnectedComponent(mask_img)
mask_labels = sitk.RelabelComponent(mask_labels)
num_lesions = int(sitk.GetArrayViewFromImage(mask_labels).max())
print(f"Nombre de lésions détectées : {num_lesions}")

# -----------------------------
# 5) Préparer les tableaux
# -----------------------------
results = []

aseg_array = sitk.GetArrayFromImage(aseg_img)
t1_array   = sitk.GetArrayFromImage(t1_img)
mask_array = sitk.GetArrayFromImage(mask_labels)

# Pour chaque lésion
for lesion_id in range(1, num_lesions+1):
    lesion_mask = mask_array == lesion_id

    # Intensité globale lésion
    lesion_voxels = t1_array[lesion_mask]
    lesion_mean   = round(lesion_voxels.mean(), 1)
    lesion_std    = round(lesion_voxels.std(), 1)
    lesion_vol    = round(lesion_mask.sum() * np.prod(t1_img.GetSpacing()), 1)

    # Intensité par zone
    zone_means = {}
    unique_zones = np.unique(aseg_array)
    for zone in unique_zones:
        if zone in exclude_labels or zone == 0:
            continue
        zone_mask = aseg_array == zone
        intersect_mask = lesion_mask & zone_mask
        if intersect_mask.sum() > 0:
            zone_intensity = round(t1_array[intersect_mask].mean(), 1)
        else:
            zone_intensity = ""
        zone_means[f"zone_{zone}_mean"] = zone_intensity

    # Ajouter ligne dans le tableau
    row = {
        "patient": "P1",
        "timepoint": "T1",
        "lesion_id": lesion_id,
        "lesion_volume_mm3": lesion_vol,
        "lesion_mean": lesion_mean,
        "lesion_std": lesion_std
    }
    row.update(zone_means)
    results.append(row)

# -----------------------------
# 6) Écrire CSV sans pandas
# -----------------------------
# Extraire tous les noms de colonnes (en ordonnant zones par numéro)
all_keys = ["patient", "timepoint", "lesion_id", "lesion_volume_mm3", "lesion_mean", "lesion_std"]
zone_keys = sorted([k for k in results[0].keys() if k.startswith("zone_")], key=lambda x: int(x.split("_")[1]))
all_keys.extend(zone_keys)

with open("P1_T1_T1_lesions_intensity.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=all_keys)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print("CSV sauvegardé : P1_T1_T1_lesions_intensity.csv")


Nombre de lésions détectées : 18
CSV sauvegardé : P1_T1_T1_lesions_intensity.csv


In [ ]:
import SimpleITK as sitk
import numpy as np
import csv

# -----------------------------
# 1) Chemins des fichiers
# -----------------------------
aseg_path = "../../27919209/MSLesSegDataset/train/P1/T1/recalage.nii.gz"
mask_path = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_MASK.nii.gz"
t1_path   = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T1.nii.gz"

# -----------------------------
# 2) Lecture des images
# -----------------------------
t1_img   = sitk.Cast(sitk.ReadImage(t1_path), sitk.sitkFloat64)
mask_img = sitk.Cast(sitk.ReadImage(mask_path) > 0, sitk.sitkUInt8)
aseg_img = sitk.ReadImage(aseg_path)

# -----------------------------
# 3) Labels ASEG à exclure
# -----------------------------
exclude_labels = [4, 43, 14, 15, 24, 31, 63]

# -----------------------------
# LUT FreeSurfer ASEG
# -----------------------------
freesurfer_aseg_labels = {
    0:  "Background",
    2:  "Left-Cerebral-White-Matter",
    3:  "Left-Cerebral-Cortex",
    4:  "Left-Lateral-Ventricle",
    5:  "Left-Inf-Lat-Vent",
    7:  "Left-Cerebellum-White-Matter",
    8:  "Left-Cerebellum-Cortex",
    10: "Left-Thalamus",
    11: "Left-Caudate",
    12: "Left-Putamen",
    13: "Left-Pallidum",
    16: "Brain-Stem",
    17: "Left-Hippocampus",
    18: "Left-Amygdala",
    26: "Left-Accumbens-area",
    28: "Left-VentralDC",
    41: "Right-Cerebral-White-Matter",
    42: "Right-Cerebral-Cortex",
    43: "Right-Lateral-Ventricle",
    44: "Right-Inf-Lat-Vent",
    46: "Right-Cerebellum-White-Matter",
    47: "Right-Cerebellum-Cortex",
    49: "Right-Thalamus",
    50: "Right-Caudate",
    51: "Right-Putamen",
    52: "Right-Pallidum",
    53: "Right-Hippocampus",
    54: "Right-Amygdala",
    58: "Right-Accumbens-area",
    60: "Right-VentralDC",
}

# -----------------------------
# 4) Identifier les lésions distinctes
# -----------------------------
mask_labels = sitk.ConnectedComponent(mask_img)
mask_labels = sitk.RelabelComponent(mask_labels)
num_lesions = int(sitk.GetArrayViewFromImage(mask_labels).max())
print(f"Nombre de lésions détectées : {num_lesions}")

# -----------------------------
# 5) Préparer les tableaux
# -----------------------------
results = []

aseg_array = sitk.GetArrayFromImage(aseg_img)
t1_array   = sitk.GetArrayFromImage(t1_img)
mask_array = sitk.GetArrayFromImage(mask_labels)

# Pour chaque lésion
for lesion_id in range(1, num_lesions+1):
    lesion_mask = mask_array == lesion_id

    # Intensité globale lésion
    lesion_voxels = t1_array[lesion_mask]
    lesion_mean   = round(lesion_voxels.mean(), 1)
    lesion_std    = round(lesion_voxels.std(), 1)
    lesion_min    = round(lesion_voxels.min(), 1)
    lesion_max    = round(lesion_voxels.max(), 1)
    lesion_range  = round(lesion_max - lesion_min, 1)

    # Moments 3 et 4 (skewness et kurtosis) sans scipy
    n = lesion_voxels.size
    if n >= 3 and lesion_std > 0:
        m3 = np.mean((lesion_voxels - lesion_mean)**3)
        m4 = np.mean((lesion_voxels - lesion_mean)**4)
        lesion_skew = round(m3 / lesion_std**3, 2)
        lesion_kurt = round(m4 / lesion_std**4 - 3, 2)
    else:
        lesion_skew = ""
        lesion_kurt = ""

    # Intensité par zone avec noms
    zone_means = {}
    unique_zones = np.unique(aseg_array)
    for zone in unique_zones:
        if zone in exclude_labels or zone == 0:
            continue
        zone_mask = aseg_array == zone
        intersect_mask = lesion_mask & zone_mask
        if intersect_mask.sum() > 0:
            zone_intensity = round(t1_array[intersect_mask].mean(), 1)
        else:
            zone_intensity = ""
        zone_name = freesurfer_aseg_labels.get(zone, f"Zone_{zone}")
        zone_means[f"{zone_name}_mean"] = zone_intensity

    # Ajouter ligne dans le tableau
    row = {
        "patient": "P1",
        "timepoint": "T1",
        "lesion_id": lesion_id,
        "lesion_volume_mm3": round(lesion_mask.sum() * np.prod(t1_img.GetSpacing()), 1),
        "lesion_mean": lesion_mean,
        "lesion_std": lesion_std,
        "lesion_min": lesion_min,
        "lesion_max": lesion_max,
        "lesion_range": lesion_range,
        "lesion_skew": lesion_skew,
        "lesion_kurt": lesion_kurt
    }
    row.update(zone_means)
    results.append(row)

# -----------------------------
# 6) Écrire CSV sans pandas
# -----------------------------
all_keys = ["patient", "timepoint", "lesion_id", "lesion_volume_mm3",
            "lesion_mean", "lesion_std", "lesion_min", "lesion_max", "lesion_range",
            "lesion_skew", "lesion_kurt"]
zone_keys = sorted([k for k in results[0].keys() if k not in all_keys])
all_keys.extend(zone_keys)

with open("P1_T1_T1_lesions_intensity_named_zones.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=all_keys)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print("CSV sauvegardé : P1_T1_T1_lesions_intensity_named_zones.csv")


Nombre de lésions détectées : 18
CSV sauvegardé : P1_T1_T1_lesions_intensity_stats.csv
